In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [2]:
data = pd.read_csv('train.csv')
print(data.head())
data.target.value_counts()
print(data.shape)

     Image         target
0   96.jpg       manipuri
1  163.jpg  bharatanatyam
2  450.jpg         odissi
3  219.jpg      kathakali
4  455.jpg         odissi
(364, 2)


In [22]:
data['labels'] = data.target.map({'mohiniyattam': 0,
                                  'odissi': 1,
                                  'kathakali': 2,
                                  'bharatanatyam': 3,
                                  'kuchipudi': 4,
                                  'sattriya': 5,
                                  'kathak': 6,
                                  'manipuri': 7})

data.head()

,Image,target,labels
0,96.jpg,manipuri,7
1,163.jpg,bharatanatyam,3
2,450.jpg,odissi,1
3,219.jpg,kathakali,2
4,455.jpg,odissi,1


In [23]:
class CustomDataset(Dataset):

    def __init__(self, data, root_dir):
        self.data = data
        self.root_dir = root_dir
        self.transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((32,32)),
                                transforms.ToTensor()])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0])
        image = io.imread(img_name)
        image = self.transform(image)
        labels = self.data.iloc[idx, 2]
        labels = np.array([labels])
        #sample = {'images': image, 'labels': labels}

        return (image, labels)

In [24]:
#transform = transforms.Compose([transforms.ToTensor()])

dataset = CustomDataset(data = data,
                        root_dir='C:/Users/Piyush/Documents/Python Scripts/Hackathons/Dance Form/train')

len(data)

364

In [25]:
total_count = len(data)

train_count = int(0.8 * total_count) 
test_count = total_count - train_count

train_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_count, test_count))

train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)  
test_dataset_loader  = torch.utils.data.DataLoader(test_dataset , batch_size=4, shuffle=False)
dataloaders = {'train': train_dataset_loader, 'test': test_dataset_loader}

364


In [40]:
print()

292


In [32]:
for batch in train_dataset_loader:
    print(batch[1])

tensor([[1],
        [1],
        [1],
        [2]])
tensor([[3],
        [0],
        [0],
        [1]])
tensor([[0],
        [0],
        [0],
        [4]])
tensor([[3],
        [1],
        [2],
        [3]])
tensor([[2],
        [4],
        [0],
        [1]])
tensor([[5],
        [5],
        [0],
        [7]])
tensor([[3],
        [5],
        [1],
        [6]])
tensor([[7],
        [1],
        [3],
        [1]])
tensor([[5],
        [7],
        [2],
        [6]])
tensor([[6],
        [0],
        [7],
        [3]])
tensor([[0],
        [4],
        [2],
        [0]])
tensor([[7],
        [3],
        [0],
        [5]])
tensor([[4],
        [6],
        [6],
        [5]])
tensor([[0],
        [0],
        [1],
        [7]])
tensor([[6],
        [1],
        [4],
        [3]])
tensor([[5],
        [4],
        [2],
        [0]])
tensor([[2],
        [1],
        [3],
        [7]])
tensor([[6],
        [2],
        [0],
        [6]])
tensor([[5],
        [6],
        [3],
       

In [27]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size = 2)
        
        self.cnn2 = nn.Conv2d(in_channels = 8, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
        self.batchnorm2 = nn.BatchNorm2d(32)
        
        self.dropout = nn.Dropout(p = 0.7)
        self.fc1 = nn.Linear(64*32, 600)
        self.fc2 = nn.Linear(600, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.cnn2(out)
        out = self.batchnorm2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = out.view(-1, 64*32)
        
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        return out

In [43]:
model = CNN()
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [46]:
from sklearn.metrics import f1_score
model.train()

epochs = 50

for epoch in range(epochs):
    correct = 0
    
    for inputs,labels in train_dataset_loader:
        
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        outputs = model(inputs)
        
        optimizer.zero_grad()
        loss = criterion(outputs, labels.squeeze())
        #loss = criterion(outputs, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        
        correct += (predicted == labels.squeeze()).sum()
        #print(correct)
    print(correct)
    print('For epoch {}/{}, training accuracy = {:.3f}%'.format(epoch+1, 10, correct*100/len(train_dataset)))
    
    model.eval()

    correct = 0
    
    for inputs, labels in test_dataset_loader:
        
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        outputs = model(inputs)
        
        _, predict = torch.max(outputs, 1)
        correct += (predict == labels).sum()
    print(correct)    
    print('For epoch {}/{}, testing accuracy = {:.3f}%'.format(epoch+1, epochs, correct*100/len(test_dataset)))

tensor(60, device='cuda:0')
For epoch 1/10, training accuracy = 20.000%
tensor(48, device='cuda:0')
For epoch 1/10, testing accuracy = 65.000%
tensor(134, device='cuda:0')
For epoch 2/10, training accuracy = 46.000%
tensor(48, device='cuda:0')
For epoch 2/10, testing accuracy = 65.000%
tensor(159, device='cuda:0')
For epoch 3/10, training accuracy = 54.000%
tensor(40, device='cuda:0')
For epoch 3/10, testing accuracy = 54.000%
tensor(161, device='cuda:0')
For epoch 4/10, training accuracy = 55.000%
tensor(49, device='cuda:0')
For epoch 4/10, testing accuracy = 67.000%
tensor(200, device='cuda:0')
For epoch 5/10, training accuracy = 68.000%
tensor(54, device='cuda:0')
For epoch 5/10, testing accuracy = 73.000%
tensor(226, device='cuda:0')
For epoch 6/10, training accuracy = 77.000%
tensor(56, device='cuda:0')
For epoch 6/10, testing accuracy = 76.000%
tensor(237, device='cuda:0')
For epoch 7/10, training accuracy = 81.000%
tensor(52, device='cuda:0')
For epoch 7/10, testing accuracy = 7